## Шаг 2. Предобработка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

В дата фреймах MOEX есть ненужные колонки где мало данных, их необходимо исключить. В дата фреймах ЦБ числовые значения из за формата ##,## нельзя перевести в float, поэтому для каждого датафрейма исключим эти проблемы.

### Импортируем получившиеся датафреймы

In [2]:
IMOEX_df = pd.read_csv('IMOEX_df')
MOEXMM_df = pd.read_csv('MOEXMM_df')
MOEXOG_df = pd.read_csv('MOEXOG_df')
GOLD_df = pd.read_csv('GOLD_df')
USDRUB_df = pd.read_csv('USDRUB_df')
INFLATION_df = pd.read_csv('INFLATION_df')

### Обработка датафрейма IMOEX

In [3]:
print(IMOEX_df.head())

  BOARDID  SECID   TRADEDATE        SHORTNAME             NAME    CLOSE  \
0    SNDX  IMOEX  2013-09-02  Индекс МосБиржи  Индекс МосБиржи  1367.53   
1    SNDX  IMOEX  2013-09-03  Индекс МосБиржи  Индекс МосБиржи  1373.82   
2    SNDX  IMOEX  2013-09-04  Индекс МосБиржи  Индекс МосБиржи  1375.66   
3    SNDX  IMOEX  2013-09-05  Индекс МосБиржи  Индекс МосБиржи  1422.40   
4    SNDX  IMOEX  2013-09-06  Индекс МосБиржи  Индекс МосБиржи  1423.40   

      OPEN     HIGH      LOW         VALUE  DURATION  YIELD  DECIMALS  \
0  1364.75  1373.73  1364.27  1.274407e+10       NaN    NaN         2   
1  1366.99  1384.18  1366.99  2.216425e+10       NaN    NaN         2   
2  1372.98  1376.42  1365.21  1.692969e+10       NaN    NaN         2   
3  1375.66  1422.40  1373.29  3.763559e+10       NaN    NaN         2   
4  1421.86  1448.75  1418.80  5.218912e+10       NaN    NaN         2   

   CAPITALIZATION CURRENCYID  DIVISOR  TRADINGSESSION  VOLUME  
0    6.096417e+12        RUB      NaN         

Посмотрим количество пропусков в каждом столбце

In [4]:
IMOEX_df.isnull().sum()

BOARDID              0
SECID                0
TRADEDATE            0
SHORTNAME            0
NAME                 0
CLOSE                0
OPEN                 0
HIGH                 0
LOW                  0
VALUE                0
DURATION          1065
YIELD             1065
DECIMALS             0
CAPITALIZATION       0
CURRENCYID           0
DIVISOR             80
TRADINGSESSION       0
VOLUME            2845
dtype: int64

In [5]:
print(IMOEX_df['DURATION'].unique())
print(IMOEX_df['YIELD'].unique())
print(IMOEX_df['DIVISOR'].unique())

[nan  0.]
[nan  0.]
[           nan 4.51876015e+09 4.51289187e+09 4.50866224e+09
 4.76294583e+09 4.63550338e+09 4.69729697e+09 4.68588832e+09
 4.66370056e+09 4.63258322e+09 4.68972910e+09 4.69700973e+09
 4.60042769e+09 4.72701074e+09 4.68707420e+09 4.78260596e+09
 4.63467912e+09 4.59665934e+09 4.59665934e+09 4.40892573e+09
 4.31169756e+09 4.31523349e+09 4.27786628e+09 4.26282623e+09
 4.14158796e+09 3.94901864e+09 3.70878002e+09 3.92779150e+09
 3.87151407e+09 4.04285118e+09 4.25783933e+09 4.80970779e+09
 4.80970779e+09 5.07923087e+09 5.07733607e+09 5.10549752e+09
 5.02319112e+09 4.94484716e+09 4.78149429e+09 4.77784852e+09
 4.93014674e+09 4.87287880e+09 3.41631366e+09 1.69197187e+09
 1.74596130e+09 1.77104960e+09 1.76843274e+09 1.76431300e+09
 1.84096879e+09 1.98085495e+09 1.97520662e+09 1.95111325e+09
 1.93733002e+09 2.16270349e+09]


<a href="https://ibb.co/ZRhMkZdt"><img src="https://i.ibb.co/bjrK8f1q/IMOEX-fin.jpg" alt="IMOEX-fin" border="0"></a>

* Признаки в DURATION и YIELD в 1065 случаях заполнены "NaN", в оставшихся - заполнены нулями.

* Признак DIVISOR - в 80 случаях заполнен "NaN". Однако его значение учитывается в расчете самого индекса, поэтому значения делителя для анализа не имеют значения.

#### Описания признаков до исключения

Для каждого из датафреймов индексов московской биржи названия столбцов совпадают, поэтому здесь я опишу каждый из столбцов до исключения каких-либо из них

* **BOARDID** - идентификатор торговой площадки и режима торгов, в данном случае торговая площадка для всех данных индекса - Московская биржа, режим торгов - SNDX - то есть индекс. Этот показатель не нужен для анализа, потому что совпадает на всех данных.

* **SECID** - уникальный идентификатор инструмента, присваиваемый биржей. В данном случае это название индекса - IMOEX, аналогично у других индексов. Нам он также не нужен для анализа, потому что совпадает для конкретного датасета у всех данных

* **TRADEDATE** - дата, на которую были получены данные. Этот столбец необходим, чтобы в дальнейшем объединить данные в один дата фрейм

* **SHORTNAME** - краткое название инструмента, в случае индексов, как правило совпадает с NAME, поэтому этот столбец не нужен

* **NAME** - полное название инструмента. Однако название инструмента выгодно поместить в заголовки тех столбцов, которые не будут удалены, чтобы обращаться к этим столбцам заранее понимая к какому индексу они относятся, поэтому мы удалим этот столбец, а имя инструмента внесем в название нужных нам столбцов

* **CLOSE** - цена закрытия - применимый к анализу столбец

* **OPEN** - цена открытия - применимый к анализу столбец

* **HIGH** - максимальная цена за время торгов - применимый к анализу столбец

* **LOW**  - минимальная цена за время торгов - применимый к анализу столбец

* **VALUE** - объем торгов - применимый к анализу столбец

* **DURATION** - в данном случае не нужен для анализа, так как термин дюрация применяется как правило к облигациям, и соответственно к облигационным индексам (RGBI), в индексах акций она не нужна.

* **YIELD** - аналогично с дюрацией - не применимый к индексу показатель (под yield имеется в виду купонная или дивидендная доходность, однако в данном случае к индексу они не применимы)

* **DECIMALS** - количество знаков после запятой при отображении цены инструмента. Этот столбец будет исключен

* **CAPITALIZATION** - оценивается по расчётному весу бумаг в итоговом значении индекса, применимый к анализу столбец

* **CURRENCYID** - валюта, в которой отображается индекс. Все индексы из всех датафреймов - российские и измеряются в рублях, поэтому этот столбец будет удален

* **DIVISOR** - это число, выбранное при создании индекса фондового рынка, взвешенного по цене, которое применяется к индексу для создания более удобного для восприятия значения индекса. 

* **TRADINGSESSION**  - акции, входящие в индекс Мосбиржи торгуются в вечернуюю сессию (то есть = 3), поэтому во всем фрейме значение столбца будет = 3. Этот столбец исключим

* **VOLUME** - значение, полностью заполненное "NaN", поэтому столбец будет исключен

In [6]:
#IMOEX

Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
IMOEX = IMOEX_df[Subset]
Headings_IMOEX = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_IMOEX',
    'OPEN': 'OPEN_IMOEX',
    'HIGH': 'HIGH_IMOEX',
    'LOW': 'LOW_IMOEX',
    'VALUE': 'VALUE_IMOEX',
    'CAPITALIZATION': 'CAPITALIZATION_IMOEX'
}
IMOEX = IMOEX.rename(columns = Headings_IMOEX)
print(IMOEX.head().to_string())
IMOEX.shape

    TRADEDATE  CLOSE_IMOEX  OPEN_IMOEX  HIGH_IMOEX  LOW_IMOEX   VALUE_IMOEX  CAPITALIZATION_IMOEX
0  2013-09-02      1367.53     1364.75     1373.73    1364.27  1.274407e+10          6.096417e+12
1  2013-09-03      1373.82     1366.99     1384.18    1366.99  2.216425e+10          6.124461e+12
2  2013-09-04      1375.66     1372.98     1376.42    1365.21  1.692969e+10          6.132675e+12
3  2013-09-05      1422.40     1375.66     1422.40    1373.29  3.763559e+10          6.341025e+12
4  2013-09-06      1423.40     1421.86     1448.75    1418.80  5.218912e+10          6.345500e+12


(2845, 7)

В получившемся итоговом датафрейме пропуском не осталось

In [7]:
IMOEX.isnull().any()

TRADEDATE               False
CLOSE_IMOEX             False
OPEN_IMOEX              False
HIGH_IMOEX              False
LOW_IMOEX               False
VALUE_IMOEX             False
CAPITALIZATION_IMOEX    False
dtype: bool

### Обработка MOEX_MM

In [8]:
print(MOEXMM_df.head())

  BOARDID   SECID   TRADEDATE                 SHORTNAME  \
0    SNDX  MOEXMM  2013-09-02  Индекс металлов и добычи   
1    SNDX  MOEXMM  2013-09-03  Индекс металлов и добычи   
2    SNDX  MOEXMM  2013-09-04  Индекс металлов и добычи   
3    SNDX  MOEXMM  2013-09-05  Индекс металлов и добычи   
4    SNDX  MOEXMM  2013-09-06  Индекс металлов и добычи   

                                NAME    CLOSE     OPEN     HIGH      LOW  \
0  Индекс МосБиржи металлов и добычи  2109.53  2103.86  2120.72  2100.19   
1  Индекс МосБиржи металлов и добычи  2102.51  2109.79  2123.56  2089.09   
2  Индекс МосБиржи металлов и добычи  2109.74  2101.98  2112.03  2088.95   
3  Индекс МосБиржи металлов и добычи  2161.35  2110.70  2162.06  2104.88   
4  Индекс МосБиржи металлов и добычи  2157.45  2159.63  2199.92  2154.78   

          VALUE  DURATION  YIELD  DECIMALS  CAPITALIZATION CURRENCYID  \
0  8.886807e+08       NaN    NaN         2    2.345484e+11        RUB   
1  1.304529e+09       NaN    NaN         2

In [9]:
IMOEX_df.isnull().sum()

BOARDID              0
SECID                0
TRADEDATE            0
SHORTNAME            0
NAME                 0
CLOSE                0
OPEN                 0
HIGH                 0
LOW                  0
VALUE                0
DURATION          1065
YIELD             1065
DECIMALS             0
CAPITALIZATION       0
CURRENCYID           0
DIVISOR             80
TRADINGSESSION       0
VOLUME            2845
dtype: int64

In [10]:
print(MOEXMM_df['DURATION'].unique())
print(MOEXMM_df['YIELD'].unique())
print(MOEXMM_df['DIVISOR'].unique())

[nan  0.]
[nan  0.]
[           nan 1.62353092e+08 1.44537357e+08 1.26491770e+08
 1.27133103e+08 1.27432336e+08 1.39922642e+08 1.34654768e+08
 1.35272929e+08 1.20986892e+08 1.12702897e+08 1.29080340e+08
 1.30446110e+08 1.34221324e+08 1.21584398e+08 1.57140921e+08
 1.59587492e+08 1.79722214e+08 1.79722214e+08 1.79575136e+08
 1.77827615e+08 1.69385852e+08 1.61823138e+08 1.69285623e+08
 1.72755994e+08 1.69563875e+08 1.79400138e+08 1.84464611e+08
 1.72138181e+08 1.58483725e+08 1.60529704e+08 1.60529704e+08
 1.77542508e+08 1.98708016e+08 2.06122011e+08 2.22341609e+08
 2.25196183e+08 2.08936150e+08 2.07648058e+08 2.12555730e+08
 2.14727821e+08 1.63382893e+08 1.00527157e+08 1.12468175e+08
 1.07058846e+08 1.08257207e+08 1.06246719e+08 1.00865142e+08
 9.96375796e+07]


In [11]:
#MOEX_MM

Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
MOEXMM = MOEXMM_df[Subset]
Headings_MOEXMM = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_MOEXMM',
    'OPEN': 'OPEN_MOEXMM',
    'HIGH': 'HIGH_MOEXMM',
    'LOW': 'LOW_MOEXMM',
    'VALUE': 'VALUE_MOEXMM',
    'CAPITALIZATION': 'CAPITALIZATION_MOEXMM'
}
MOEXMM = MOEXMM.rename(columns = Headings_MOEXMM)
print(MOEXMM.head().to_string())
MOEXMM.shape

    TRADEDATE  CLOSE_MOEXMM  OPEN_MOEXMM  HIGH_MOEXMM  LOW_MOEXMM  VALUE_MOEXMM  CAPITALIZATION_MOEXMM
0  2013-09-02       2109.53      2103.86      2120.72     2100.19  8.886807e+08           2.345484e+11
1  2013-09-03       2102.51      2109.79      2123.56     2089.09  1.304529e+09           2.337673e+11
2  2013-09-04       2109.74      2101.98      2112.03     2088.95  1.283077e+09           2.345715e+11
3  2013-09-05       2161.35      2110.70      2162.06     2104.88  2.779851e+09           2.403100e+11
4  2013-09-06       2157.45      2159.63      2199.92     2154.78  3.672785e+09           2.398759e+11


(2842, 7)

In [12]:
MOEXMM.isnull().any()

TRADEDATE                False
CLOSE_MOEXMM             False
OPEN_MOEXMM              False
HIGH_MOEXMM              False
LOW_MOEXMM               False
VALUE_MOEXMM             False
CAPITALIZATION_MOEXMM    False
dtype: bool

### Обработка датафрейма с данными по инфляции

In [13]:
print(INFLATION_df.head().to_string())

         Date Key_rate Inflation Inflation_goal
0  2025-01-01    21,00      9,92           4,00
1  2024-12-01    21,00      9,52           4,00
2  2024-11-01    21,00      8,88           4,00
3  2024-10-01    21,00      8,54           4,00
4  2024-09-01    19,00      8,63           4,00


### Описание признаков до исключения

* **Key_rate** - ключевая ставка, устанавливаемая ЦБ. 

* **Inflation** - среднемесячный темп инфляции. Этот признак особо важен, так как золото является хеджирующим активом от инфляции, следовательно, именно этот показатель будет формировать основную динамику цен на золото

* **Inflational_goal** - таргет по инфляции 

В столбцах Key_rate, Inflation, Inflation_goal значения нельзя конвертировать во float, так как там стоит запятая как разделитель. Реализуем функцию которая меняет формат этих данных

In [14]:
def correct(number):
    cor_val = number.replace(',', '.')
    return cor_val
corrected_1 = INFLATION_df['Key_rate'].map(correct)
INFLATION_df['Key_rate'] = corrected_1
corrected_2 = INFLATION_df['Inflation'].map(correct)
INFLATION_df['Inflation'] = corrected_2
corrected_3 = INFLATION_df['Inflation_goal'].map(correct)
INFLATION_df['Inflation_goal'] = corrected_3
print(INFLATION_df.head().to_string())


         Date Key_rate Inflation Inflation_goal
0  2025-01-01    21.00      9.92           4.00
1  2024-12-01    21.00      9.52           4.00
2  2024-11-01    21.00      8.88           4.00
3  2024-10-01    21.00      8.54           4.00
4  2024-09-01    19.00      8.63           4.00


In [15]:
print(INFLATION_df['Inflation_goal'].unique())

['4.00' '—']


Таргет по инфляции одинаковый для всего дата фрейма, ниже 2014 года он заполнен NaN, так как тогда еще не проводилась политика таргетирования инфляции, поэтому исключим этот столбец

In [16]:
INFLATION = INFLATION_df.drop(['Inflation_goal'], axis = 1)
INFLATION.rename(columns = {'Date': 'TRADEDATE'},  inplace=True)
INFLATION.isnull().any()

TRADEDATE    False
Key_rate     False
Inflation    False
dtype: bool

In [17]:
print(INFLATION.head().to_string())
INFLATION.shape

    TRADEDATE Key_rate Inflation
0  2025-01-01    21.00      9.92
1  2024-12-01    21.00      9.52
2  2024-11-01    21.00      8.88
3  2024-10-01    21.00      8.54
4  2024-09-01    19.00      8.63


(137, 3)

### Обработка датафрейма валютного курса USD RUB из данных ЦБ

Признаком является официальный курсвалютной пары доллар рубль Банка России

In [18]:
print(USDRUB_df.head().to_string())

         Date  USD_RUB
0  03.09.2013  33.2522
1  04.09.2013  33.3693
2  05.09.2013  33.4656
3  06.09.2013  33.3901
4  07.09.2013  33.4338


Единственной проблемой в этом датафрейме является тот факт, что дата не находится в формате date.


In [19]:
USDRUB_df['Date'] = pd.to_datetime(USDRUB_df['Date'], format='%d.%m.%Y')
USDRUB_df.rename(columns = {'Date': 'TRADEDATE'},  inplace=True)
print(USDRUB_df.head().to_string())
USDRUB_df.shape

   TRADEDATE  USD_RUB
0 2013-09-03  33.2522
1 2013-09-04  33.3693
2 2013-09-05  33.4656
3 2013-09-06  33.3901
4 2013-09-07  33.4338


(2800, 2)

In [20]:
USDRUB_df.isnull().any()

TRADEDATE    False
USD_RUB      False
dtype: bool

### Обработка датафрейма цен на золото из данных ЦБ

In [21]:
print(GOLD_df.head().to_string())

         Date Price_GOLD
0  03.09.2013    1487,36
1  04.09.2013     1492,6
2  05.09.2013    1510,36
3  06.09.2013    1494,07
4  07.09.2013    1470,76


In [22]:
GOLD_df.isnull().any()

Date          False
Price_GOLD    False
dtype: bool

In [23]:
GOLD_df['Date'] = pd.to_datetime(GOLD_df['Date'], format='%d.%m.%Y')
GOLD_df.rename(columns = {'Date': 'TRADEDATE'},  inplace=True)
print(GOLD_df.head().to_string())

   TRADEDATE Price_GOLD
0 2013-09-03    1487,36
1 2013-09-04     1492,6
2 2013-09-05    1510,36
3 2013-09-06    1494,07
4 2013-09-07    1470,76


In [24]:
def correct(number):
    cor_val = number.replace(',', '.')
    return cor_val
corrected = GOLD_df['Price_GOLD'].map(correct)
GOLD_df['Price_GOLD'] = corrected
print(GOLD_df.head().to_string())

   TRADEDATE Price_GOLD
0 2013-09-03    1487.36
1 2013-09-04     1492.6
2 2013-09-05    1510.36
3 2013-09-06    1494.07
4 2013-09-07    1470.76


### Объединение датафреймов

1) Не все данные отсортированы по дате: GOLD_df USDRUB_df, MOEXMM, IMOEX отсортированы от start_date, датафрейм инфляции, наоборот, начинается с end_date. Поэтому нам надо отсортировать INFLATION по дате

In [25]:
INFLATION= INFLATION.sort_values(by='TRADEDATE')
print(INFLATION)

      TRADEDATE Key_rate Inflation
136  2013-09-01     5.50      6.14
135  2013-10-01     5.50      6.27
134  2013-11-01     5.50      6.50
133  2013-12-01     5.50      6.47
132  2014-01-01     5.50      6.07
..          ...      ...       ...
4    2024-09-01    19.00      8.63
3    2024-10-01    21.00      8.54
2    2024-11-01    21.00      8.88
1    2024-12-01    21.00      9.52
0    2025-01-01    21.00      9.92

[137 rows x 3 columns]


In [ ]:
print(IMOEX['TRADEDATE'].duplicated().sum())  # Проверка дубликатов в IMOEX
print(MOEXMM['TRADEDATE'].duplicated().sum())  # Проверка дубликатов в MOEXMM

0
0


In [ ]:
# во избежание ошибок еще раз перевод столбца TRADEDATE в формат дат
IMOEX['TRADEDATE'] = pd.to_datetime(IMOEX['TRADEDATE'])
MOEXMM['TRADEDATE'] = pd.to_datetime(MOEXMM['TRADEDATE'])
INFLATION['TRADEDATE'] = pd.to_datetime(INFLATION['TRADEDATE'])
USDRUB_df['TRADEDATE'] = pd.to_datetime(USDRUB_df['TRADEDATE'])
GOLD_df['TRADEDATE'] = pd.to_datetime(GOLD_df['TRADEDATE'])

dataframes = [IMOEX, MOEXMM, INFLATION, USDRUB_df, GOLD_df]

# функция для объединения (функция reduce из модуля functools последовательно кумулятивно берет элементы из dataframes и объединяет их слева направо)
RESULT_df = reduce(lambda x, y: pd.merge(x, y, on='TRADEDATE', how='outer'), dataframes)

print(RESULT_df.head(25).to_string())
RESULT_df.shape

    TRADEDATE  CLOSE_IMOEX  OPEN_IMOEX  HIGH_IMOEX  LOW_IMOEX   VALUE_IMOEX  CAPITALIZATION_IMOEX  CLOSE_MOEXMM  OPEN_MOEXMM  HIGH_MOEXMM  LOW_MOEXMM  VALUE_MOEXMM  CAPITALIZATION_MOEXMM Key_rate Inflation  USD_RUB Price_GOLD
0  2013-09-01          NaN         NaN         NaN        NaN           NaN                   NaN           NaN          NaN          NaN         NaN           NaN                    NaN     5.50      6.14      NaN        NaN
1  2013-09-02      1367.53     1364.75     1373.73    1364.27  1.274407e+10          6.096417e+12       2109.53      2103.86      2120.72     2100.19  8.886807e+08           2.345484e+11      NaN       NaN      NaN        NaN
2  2013-09-03      1373.82     1366.99     1384.18    1366.99  2.216425e+10          6.124461e+12       2102.51      2109.79      2123.56     2089.09  1.304529e+09           2.337673e+11      NaN       NaN  33.2522    1487.36
3  2013-09-04      1375.66     1372.98     1376.42    1365.21  1.692969e+10          6.132675e+1

### Заполнение пропусков

In [28]:
# смотрим количество пропусков
RESULT_df.isnull().sum()

TRADEDATE                   0
CLOSE_IMOEX               639
OPEN_IMOEX                639
HIGH_IMOEX                639
LOW_IMOEX                 639
VALUE_IMOEX               639
CAPITALIZATION_IMOEX      639
CLOSE_MOEXMM              642
OPEN_MOEXMM               642
HIGH_MOEXMM               642
LOW_MOEXMM                642
VALUE_MOEXMM              642
CAPITALIZATION_MOEXMM     642
Key_rate                 3347
Inflation                3347
USD_RUB                   684
Price_GOLD                684
dtype: int64

Если в день нет данных по индексам, это означает что день - выходной. В данном случае разумно было бы предположить, что цена за выходной будет средним между ценами в смежные рабочие дни, поэтому эти пропуски мы заполним средним значением. Также составим новую колонку, где будет прописано рабочий это или выходной день.

In [29]:
print(RESULT_df.dtypes)

TRADEDATE                datetime64[ns]
CLOSE_IMOEX                     float64
OPEN_IMOEX                      float64
HIGH_IMOEX                      float64
LOW_IMOEX                       float64
VALUE_IMOEX                     float64
CAPITALIZATION_IMOEX            float64
CLOSE_MOEXMM                    float64
OPEN_MOEXMM                     float64
HIGH_MOEXMM                     float64
LOW_MOEXMM                      float64
VALUE_MOEXMM                    float64
CAPITALIZATION_MOEXMM           float64
Key_rate                         object
Inflation                        object
USD_RUB                         float64
Price_GOLD                       object
dtype: object


In [30]:
RESULT_df['Price_GOLD'] = pd.to_numeric(RESULT_df['Price_GOLD'])
RESULT_df['Key_rate'] = pd.to_numeric(RESULT_df['Key_rate'])
RESULT_df['Inflation'] = pd.to_numeric(RESULT_df['Inflation'])
print(RESULT_df.dtypes)

TRADEDATE                datetime64[ns]
CLOSE_IMOEX                     float64
OPEN_IMOEX                      float64
HIGH_IMOEX                      float64
LOW_IMOEX                       float64
VALUE_IMOEX                     float64
CAPITALIZATION_IMOEX            float64
CLOSE_MOEXMM                    float64
OPEN_MOEXMM                     float64
HIGH_MOEXMM                     float64
LOW_MOEXMM                      float64
VALUE_MOEXMM                    float64
CAPITALIZATION_MOEXMM           float64
Key_rate                        float64
Inflation                       float64
USD_RUB                         float64
Price_GOLD                      float64
dtype: object


In [45]:
numeric_columns = RESULT_df.select_dtypes(include=['float64', 'int64']).columns
print(numeric_columns)

Index(['CLOSE_IMOEX', 'OPEN_IMOEX', 'HIGH_IMOEX', 'LOW_IMOEX', 'VALUE_IMOEX',
       'CAPITALIZATION_IMOEX', 'CLOSE_MOEXMM', 'OPEN_MOEXMM', 'HIGH_MOEXMM',
       'LOW_MOEXMM', 'VALUE_MOEXMM', 'CAPITALIZATION_MOEXMM', 'Key_rate',
       'Inflation', 'USD_RUB', 'Price_GOLD'],
      dtype='object')


In [ ]:
RESULT_df[numeric_columns] = RESULT_df[numeric_columns].interpolate(method='linear')
mis_replacer = SimpleImputer(strategy="mean") 
RESULT_df[numeric_columns] = mis_replacer.fit_transform(RESULT_df[numeric_columns])
print(RESULT_df.head(10).to_string())

   TRADEDATE  CLOSE_IMOEX   OPEN_IMOEX   HIGH_IMOEX    LOW_IMOEX   VALUE_IMOEX  CAPITALIZATION_IMOEX  CLOSE_MOEXMM  OPEN_MOEXMM  HIGH_MOEXMM  LOW_MOEXMM  VALUE_MOEXMM  CAPITALIZATION_MOEXMM  Key_rate  Inflation    USD_RUB   Price_GOLD
0 2013-09-01  2427.320151  2427.045428  2445.449308  2407.162077  5.210899e+10          9.884176e+12   6293.904813  6294.929209   6342.64829  6240.54035  9.512234e+09           1.025128e+12       5.5      6.140  66.741709  3504.222523
1 2013-09-02  1367.530000  1364.750000  1373.730000  1364.270000  1.274407e+10          6.096417e+12   2109.530000  2103.860000   2120.72000  2100.19000  8.886807e+08           2.345484e+11       5.5      6.145  66.741709  3504.222523
2 2013-09-03  1373.820000  1366.990000  1384.180000  1366.990000  2.216425e+10          6.124461e+12   2102.510000  2109.790000   2123.56000  2089.09000  1.304529e+09           2.337673e+11       5.5      6.150  33.252200  1487.360000
3 2013-09-04  1375.660000  1372.980000  1376.420000  1365.21

In [35]:
RESULT_df.isnull().any()

TRADEDATE                False
CLOSE_IMOEX              False
OPEN_IMOEX               False
HIGH_IMOEX               False
LOW_IMOEX                False
VALUE_IMOEX              False
CAPITALIZATION_IMOEX     False
CLOSE_MOEXMM             False
OPEN_MOEXMM              False
HIGH_MOEXMM              False
LOW_MOEXMM               False
VALUE_MOEXMM             False
CAPITALIZATION_MOEXMM    False
Key_rate                 False
Inflation                False
USD_RUB                  False
Price_GOLD               False
dtype: bool

In [47]:
RESULT_df.to_csv('RESULT_df')